In [80]:
# custom modules
from ranking_score import *
from iaa import *

In [81]:
import pandas as pd
pd.set_option("display.max_rows", None)

import numpy as np
from koMecab import mecab
import sacrebleu

### Get the data

In [82]:
before = pd.read_csv("../raw_data/relative_ranking_a+b_before.csv", sep="\t", encoding="utf-8")
after = pd.read_csv("../raw_data/relative_ranking_a+b_after.csv", sep="\t", encoding="utf-8")

In [83]:
del before["Unnamed: 0"]
del after["Unnamed: 0"]
before.head(1)

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error
0,1,"Royal news: How Princess Beatrice ""reacted with fury at hostile family members""","영국 왕실 뉴스: 베아트리스 공주, ‘적대적인 왕실 구성원들에게 이렇게 분노 표출해’","왕실 뉴스: 베아트리체 공주가 ""적대하는 가족들에게 분노의 반응을 보였다""",왕실 뉴스: 베아트리체 공주가 적대적인 가족들에게 분노로 어떻게 반응했는가,2,2,1,F


### Get the result

In [6]:
result_before = get_result(before)
result_after = get_result(after)

In [178]:
result_before

,MT_Y,MT_Z
MT_better,61,56
HT_better,256,284
Tie,120,97
P-value,3.62841e-21,5.10358e-29
Valid,True,True
Ab_score,2.22883,2.07551


In [179]:
result_after

,MT_Y,MT_Z
MT_better,49,40
HT_better,290,321
Tie,98,76
P-value,3.71747e-32,3.81629e-44
Valid,True,True
Ab_score,2.13959,1.93593


#### absolute score of HT

In [181]:
absolute_score(before, "Ref")

{'Ref': (2.7734553775743707, 360, 55, 22)}

In [182]:
absolute_score(after, "Ref")

{'Ref': (2.8260869565217392, 380, 38, 19)}

### Result: Error=="T"

In [170]:
before_error = before[before["Error"] == "T"]
len(before_error)

92

In [171]:
after_error = after[after["Error"] == "T"]
len(after_error)

92

In [183]:
result_before_error = get_result(before_error)
result_after_error = get_result(after_error)

In [184]:
result_before_error

,MT_Y,MT_Z
MT_better,18,12
HT_better,58,59
Tie,16,21
P-value,3.67005e-05,5.34814e-07
Valid,True,True
Ab_score,2.18478,2.09783


In [185]:
result_after_error

,MT_Y,MT_Z
MT_better,9,6
HT_better,61,68
Tie,22,18
P-value,4.60625e-08,3.2624e-11
Valid,True,True
Ab_score,2.11957,1.94565


#### absolute score of HT

In [187]:
absolute_score(before_error, "Ref")

{'Ref': (2.7065217391304346, 74, 9, 9)}

In [186]:
absolute_score(after_error, "Ref")

{'Ref': (2.858695652173913, 82, 7, 3)}

### IAA

In [10]:
file_dicts = read_files()  # group, dfs
individual_IAA(file_dicts, "a")

0.4759

In [199]:
file_dicts.keys()

dict_keys(['a1', 'a2', 'b1', 'b2', 'c1', 'c2', 'd1', 'd2', 'e1', 'e2', 'f1', 'f2', 'g1', 'g2', 'h1', 'h2', 'i1', 'i2', 'j1', 'j2', 'k1', 'k2'])

In [11]:
final_IAA(file_dicts)

{'a': 0.4759,
 'b': 0.3032,
 'c': 0.4107,
 'd': 0.1687,
 'e': 0.2711,
 'f': 0.382,
 'g': 0.3773,
 'h': 0.1314,
 'i': 0.2233,
 'j': 0.1971,
 'k': 0.474}

In [16]:
kappa_dict = final_IAA(file_dicts)
print(np.mean([score for _, score in kappa_dict.items()]))
print(np.min([score for _, score in kappa_dict.items()]))
print(np.max([score for _, score in kappa_dict.items()]))

0.3104272727272727
0.1314
0.4759


### SacreBLEU

#### before

In [121]:
%%time
reference = before.Ref.tolist()
hypothesis_y = before.MT_Y.tolist()
hypothesis_z = before.MT_Z.tolist()

reference = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in reference]
hypothesis_y = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in hypothesis_y]
hypothesis_z = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in hypothesis_z]

for system in [hypothesis_y, hypothesis_z]:
    
    bleu = sacrebleu.corpus_bleu(system, [reference])
    ter = sacrebleu.corpus_ter(system, [reference])
    !--force
    chrf = sacrebleu.corpus_chrf(system, [reference])
    
    if system == hypothesis_y:
        print("[MT_Y]")
    else:
        print("[MT_Z]")
        
    print("BLEU: ", bleu.score)
    print("TER: ", ter.score)
    print("chrF", chrf.score)
    print()

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


[MT_Y]
BLEU:  19.785266218415302
TER:  0.6547766323024055
chrF 0.2707334386314116

/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option
[MT_Z]
BLEU:  20.19309663896737
TER:  0.6626804123711341
chrF 0.26968267974474885

CPU times: user 43.2 s, sys: 290 ms, total: 43.5 s
Wall time: 43.7 s


#### after

In [120]:
%%time
reference = after.Ref.tolist()
hypothesis_y = after.MT_Y.tolist()
hypothesis_z = after.MT_Z.tolist()

reference = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in reference]
hypothesis_y = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in hypothesis_y]
hypothesis_z = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in hypothesis_z]

for system in [hypothesis_y, hypothesis_z]:
    
    bleu = sacrebleu.corpus_bleu(system, [reference])
    ter = sacrebleu.corpus_ter(system, [reference])
    !--force
    chrf = sacrebleu.corpus_chrf(system, [reference])
    
    if system == hypothesis_y:
        print("[MT_Y]")
    else:
        print("[MT_Z]")
        
    print("BLEU: ", bleu.score)
    print("TER: ", ter.score)
    print("chrF", chrf.score)
    print()

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


[MT_Y]
BLEU:  19.77921498933438
TER:  0.6520962199312714
chrF 0.271264889552056

/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option
[MT_Z]
BLEU:  20.241028636714525
TER:  0.6623367697594502
chrF 0.2709943645290349

CPU times: user 42.3 s, sys: 334 ms, total: 42.6 s
Wall time: 42.8 s


#### Google Translate

In [84]:
df = pd.read_csv("../data_out/_google+ter.csv", sep="\t", encoding="utf-8")
del df["Unnamed: 0"]
df.head(1)

,Google Before,Google After,TER
0,"왕실 뉴스: 베아트리체 공주가 어떻게 ""적대적인 가족 구성원들에게 분노에 반응했는지""",Royal News : 공주님이 적대 가족 구성원들에게 분노와 반응 한 공주,0.9091


In [85]:
len(df)

437

In [168]:
reference = after.Ref.tolist()
google_before = df["Google Before"].tolist()
google_after = df["Google After"].tolist()

reference = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in reference]
google_before = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in google_before]
google_after = [' '.join(mecab(s)).replace("▃", "").replace("  ", " ") for s in google_after]

for case in [google_before, google_after]:
    
    bleu = sacrebleu.corpus_bleu(case, [reference])
    ter = sacrebleu.corpus_ter(case, [reference])
    !--force
    chrf = sacrebleu.corpus_chrf(case, [reference])
    
    if case == google_before:
        print("[Google Before]")
    else:
        print("[Google After]")
        
    print("BLEU: ", bleu.score)
    print("TER: ", ter.score)
    print("chrF", chrf.score)
    print()

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


[Google Before]
BLEU:  13.117676138341224
TER:  0.7595189003436427
chrF 0.19496856536087329

/bin/bash: --: invalid option
Usage:	/bin/bash [GNU long option] [option] ...
	/bin/bash [GNU long option] [option] script-file ...
GNU long options:
	--debug
	--debugger
	--dump-po-strings
	--dump-strings
	--help
	--init-file
	--login
	--noediting
	--noprofile
	--norc
	--posix
	--rcfile
	--restricted
	--verbose
	--version
Shell options:
	-ilrsD or -c command or -O shopt_option		(invocation only)
	-abefhkmnptuvxBCHP or -o option
[Google After]
BLEU:  9.492515518561063
TER:  0.8156701030927835
chrF 0.1588497884299075



### Qualitative Analysis with TER

In [86]:
mty = pd.read_csv("../data_out/_mtY_ter.csv", sep="\t", encoding="utf-8")
mtz = pd.read_csv("../data_out/_mtZ_ter.csv", sep="\t", encoding="utf-8")

In [117]:
mty[mty.TER >= 0.8]

,Unnamed: 0,Segment ID,Before_tokens,After_tokens,TER
225,225,226,""" 만약 내 가 당신 을 위해 하 는 일 을 계속 하 려고 한다면 , 나 는 나 자신 을 해칠 것 입니다 .",내 가 널 위해 계속 노력 한다면 나 자신 을 해칠 거 야 .,1.0714
298,298,299,""" 주립 대학 노동자 들 "" 이 가장 압박 하 는 불만 사항 은 낮 은 임금 과 직장 에서 의 품위 부족 에 집중 되 어 있 다 .",주립 대 근로자 들 의 가장 큰 불만 은 저 임금 과 직장 의 품위 저하 에 있 다 .,0.8000


In [118]:
mtz[mtz.TER >= 0.8]

,Unnamed: 0,Segment ID,Before_tokens,After_tokens,TER
345,345,346,"일 하 는 엄마 로서 아기 와 함께 여행 하 는 것 , 세상 에 , 정말 많 은 일 이 긴 하 지만 , 모든 것 이 너무 흥미진진 해 .","일 하 는 엄마 로서 아기 와 함께 여행 하 는 것 도 좋 지만 , 정말 신 나 . """,0.8571


In [121]:
before[before["Segment ID"] == 346]

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error
345,346,"Being a working mum and travelling as well with a baby, my goodness it's a lot, but it's all so exciting.","워킹맘이 되고 아기와 돌아다니는 건 세상에나 정말 벅차다, 그러지만 둘다 정말 즐겁다.”","일하는 엄마로서, 아이와 함께 여행한다는 것, 세상에, 많은 일이지만, 모든 것이 너무 신나요.","일하는 엄마로서 아기와 함께 여행하는 것, 세상에, 정말 많은 일이긴 하지만, 모든 것이 너무 흥미진진해.",1,2,2,T


In [122]:
after[after["Segment ID"] == 346]

,Segment ID,Source Segment,Ref,MT_Y,MT_Z,Ref_rating,MT_Y_rating,MT_Z_rating,Error
345,346,"Being a working mum and travelling as well with a baby, my goodness it's a lot, but it's all so exciting.”","워킹맘이 되고 아기와 돌아다니는 건 세상에나 정말 벅차다, 그러지만 둘다 정말 즐겁다.”","일하는 엄마로서, 그리고 아이와 함께 여행한다는 것, 정말 많은 일이지만, 모든 것이 너무 즐겁습니다.""","일하는 엄마로서 아기와 함께 여행하는 것도 좋지만, 정말 신나.""",1,1,2,T
